Neste notebook será criado um perceptron de uma camada para aprender a função AND.

Tudo será feito manualmente usando o pytorch para operações de tensores.

In [1]:
import torch
from torch import nn

import numpy as np

In [2]:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])

X

array([[0, 0],
       [0, 1],
       [1, 0],
       [1, 1]])

In [3]:
#Converte X para um tensor para o pytorch
X_t = torch.from_numpy(X).double()

X_t

tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]], dtype=torch.float64)

In [4]:
Y = np.array([[0], [0], [0], [1]])

Y

array([[0],
       [0],
       [0],
       [1]])

In [5]:
Y_t = torch.from_numpy(Y).double()

Y_t

tensor([[0.],
        [0.],
        [0.],
        [1.]], dtype=torch.float64)

Configura-se o pytorch para usar acelerção em GPU:

In [6]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")


Using cuda device


Com o dataset pronto, começa-se a definição da rede neural:

In [7]:
W = torch.zeros((2, 1), dtype=torch.float64)

W

tensor([[0.],
        [0.]], dtype=torch.float64)

In [8]:
soma = torch.matmul(X_t, W)

soma

tensor([[0.],
        [0.],
        [0.],
        [0.]], dtype=torch.float64)

In [9]:
#Retorna 0 se x < 1, ou 1 se x >= 1.
def step(x):
    x = torch.as_tensor(x, dtype=torch.float64)  # garante tensor
    return torch.ge(x, 1).to(torch.float64)

step(0)

tensor(0., dtype=torch.float64)

In [10]:
ativacao = step(soma)

In [11]:
erro = Y_t - ativacao

erro

tensor([[0.],
        [0.],
        [0.],
        [1.]], dtype=torch.float64)

In [12]:
X_transposed = X_t.t()

X_transposed

tensor([[0., 0., 1., 1.],
        [0., 1., 0., 1.]], dtype=torch.float64)

In [13]:
delta = torch.matmul(X_transposed, erro)
delta

tensor([[1.],
        [1.]], dtype=torch.float64)

In [14]:
learning_rate = 0.1

treinamento = delta * learning_rate

treinamento

tensor([[0.1000],
        [0.1000]], dtype=torch.float64)

In [15]:
momento = 1
treinamento_ = W * momento + treinamento

treinamento

tensor([[0.1000],
        [0.1000]], dtype=torch.float64)

In [16]:
W = treinamento

W

tensor([[0.1000],
        [0.1000]], dtype=torch.float64)

Inicio do treinamento

In [17]:
epoch = 0
learning_rate = 0.1
momento = 1

for i in range(15):
    epoch += 1
    
    soma = torch.matmul(X_t, W)
    ativacao = step(soma)
    erro = Y_t - ativacao
    X_transposed = X_t.t()
    delta = torch.matmul(X_transposed, erro)
    treinamento = W * momento + delta * learning_rate
    W = treinamento

    
    errors_sum = torch.sum(erro).item()
    print('Epoca:', epoch, ' Erro: ', errors_sum)

    if errors_sum == 0.0:
        break
    
    

Epoca: 1  Erro:  1.0
Epoca: 2  Erro:  1.0
Epoca: 3  Erro:  1.0
Epoca: 4  Erro:  1.0
Epoca: 5  Erro:  0.0


Teste da rede neural:

In [18]:
teste_sum = torch.matmul(X_t, W)
teste_ativacao = step(teste_sum)

teste_ativacao

tensor([[0.],
        [0.],
        [0.],
        [1.]], dtype=torch.float64)

In [19]:
def perceptron(x, w):
    teste_sum = torch.matmul(x, w)
    return step(teste_sum)

In [20]:
for value in X_t:
    print("A entrada: " + str (value.numpy()) + ", tem output: " + str(perceptron(value, W).numpy()))

A entrada: [0. 0.], tem output: [0.]
A entrada: [0. 1.], tem output: [0.]
A entrada: [1. 0.], tem output: [0.]
A entrada: [1. 1.], tem output: [1.]


Agora farei o mesmo processo, porém usando a biblioteca pytorch para fazer tudo:

In [ ]:
class Perceptron(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        
        #Terá apenas duas camadas, uma com in_features neuronios de entrada e outra 1 neuronio de saída.
        self.linear = nn.Linear(in_features, 1)

    def forward(self, x):
        return self.linear(x)                     # depois aplicamos função de ativação se quiser

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Carrega X_t e Y_t no device (GPU)
X_t = torch.from_numpy(X).float().to(device)          # (N, in_features)
Y_t = torch.from_numpy(Y).float().view(-1, 1).to(device)  #.view(-1, 1) converte o tensor para (N, 1)

model = Perceptron(in_features=X_t.shape[1]).to(device)

# Para função loss
criterion = nn.BCEWithLogitsLoss()    # para classificação binária (0/1)

optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [23]:
num_epochs = 1000

for epoch in range(num_epochs):
    
    # forward
    logits = model(X_t)
    loss = criterion(logits, Y_t)

    # backward + update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 1 == 0:
        print(epoch+1, loss.item())

1 0.6322481632232666
2 0.6296948194503784
3 0.6271647214889526
4 0.6246574521064758
5 0.6221727132797241
6 0.6197099089622498
7 0.6172687411308289
8 0.614848792552948
9 0.6124497652053833
10 0.6100713014602661
11 0.6077129244804382
12 0.6053743958473206
13 0.603055477142334
14 0.6007556319236755
15 0.5984748005867004
16 0.5962125062942505
17 0.5939686894416809
18 0.5917428135871887
19 0.5895347595214844
20 0.5873443484306335
21 0.5851711630821228
22 0.5830151438713074
23 0.5808759331703186
24 0.5787532925605774
25 0.5766471028327942
26 0.5745570659637451
27 0.5724830627441406
28 0.5704247951507568
29 0.5683821439743042
30 0.5663548707962036
31 0.5643427968025208
32 0.5623457431793213
33 0.5603636503219604
34 0.55839604139328
35 0.5564430952072144
36 0.55450439453125
37 0.5525799989700317
38 0.5506695508956909
39 0.5487729907035828
40 0.5468901991844177
41 0.5450209379196167
42 0.5431650876998901
43 0.5413225293159485
44 0.539493203163147
45 0.5376767516136169
46 0.5358732342720032
47 0

In [24]:
with torch.no_grad():
    for value in X_t:

        logits = model(value)
        preds = step(logits)

        # Passa o tensor preds para a cpu, e pega o campo de valor dela.
        print("A entrada: " + str (value.cpu().numpy()) + ", tem output: " + str(preds.cpu().numpy()))

A entrada: [0. 0.], tem output: [0.]
A entrada: [0. 1.], tem output: [0.]
A entrada: [1. 0.], tem output: [0.]
A entrada: [1. 1.], tem output: [1.]
